In [69]:
#---Step 0: Import necessary libraries and render plots inline---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('fivethirtyeight')

In [70]:
df = pd.read_excel("/Users/JCheong/sales-funnel.xlsx")

df

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [71]:
# Define the status column as a category and set the order that we want to view
df["Status"] = df["Status"].astype("category")

df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

# Pivot the Data

We'll build up understanding the pivot table functionality one step at a time

The simplest pivot table must have a dataframe and an index. In this case let's use the Name as our index.

In [72]:
tester = pd.pivot_table(df, index=["Name"])

# Test sorting values
# tester.sort_values(["Quantity"], ascending=False)

tester

,Account,Price,Quantity
Name,,,
Barton LLC,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",737550.0,35000.0,1.000000
Herman LLC,141962.0,65000.0,2.000000
Jerde-Hilpert,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",307599.0,7000.0,3.000000
Keeling LLC,688981.0,100000.0,5.000000
Kiehn-Spinka,146832.0,65000.0,2.000000
Koepp Ltd,729833.0,35000.0,2.000000
Kulas Inc,218895.0,25000.0,1.500000


We can have multiple indexes as well - most of the pivot_table arguments can take multiple values as a list.

In [73]:
pd.pivot_table(df, index=["Manager", "Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

You can see that the pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. Now we start to get a glimpse of what a pivot table can do for us.

For this purpose, the Account and Quantity columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the values field.

In [74]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

The price column automatically averages the data but we can do a count or a sum. Adding them is simple using aggfunc and np.sum .

In [75]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

aggfunc can take a list of functions. Let’s try a mean using the numpy mean function and len to get a count.

In [76]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean, len])

# Alternative that uses functions to apply using double quotes
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=["mean", "count"])

mean count
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

# Adding Columns

If we want to see sales broken down by products (along the columns), we can use the columns variable to define one or more columns.

I think one of the confusing points with the pivot_table is the use of columns and values . Remember, columns are optional - they provide an additional way to segment the actual values you care about. The aggregation functions are applied to the values you list.

In [77]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], columns=["Product"], aggfunc=["sum"])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

The NaN’s are a bit distracting. If we want to remove them, we could use fill_value to set them to 0.

In [78]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], columns=["Product"], aggfunc=["sum"], fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

It would also be helpful to add the quantity in addition to the price

In [79]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price", "Quantity"], columns=["Product"], aggfunc=["sum"], fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

What’s interesting is that you can move items to the index to get a different visual representation. Remove Product from the columns and add to the index .

In [80]:
pd.pivot_table(df, index=["Manager", "Rep", "Product"], values=["Price", "Quantity"], aggfunc=["sum"], fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [81]:
tester = pd.pivot_table(df, index=["Manager", "Rep", "Product"], values=["Price", "Quantity"], aggfunc=["sum"], fill_value=0)

tester

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [82]:
# Allows us to drop a level if we want to do calculations without worrying about a multi-index column index
tester.columns = tester.columns.droplevel(level=0)
tester["Average_Price"] = tester["Price"] / tester["Quantity"]

tester

Price  Quantity  Average_Price
Manager       Rep           Product                                     
Debra Henley  Craig Booker  CPU           65000         2   32500.000000
                            Maintenance    5000         2    2500.000000
                            Software      10000         1   10000.000000
              Daniel Hilton CPU          105000         4   26250.000000
                            Software      10000         1   10000.000000
              John Smith    CPU           35000         1   35000.000000
                            Maintenance    5000         2    2500.000000
Fred Anderson Cedric Moss   CPU           95000         3   31666.666667
                            Maintenance    5000         1    5000.000000
                            Software      10000         1   10000.000000
              Wendy Yule    CPU          165000         7   23571.428571
                            Maintenance    7000         3    2333.333333
                            Monitor        5000         2    2500.000000

In [83]:
pd.pivot_table(df, index=["Manager", "Rep", "Product"], values=["Price", "Quantity"], aggfunc=["sum", "mean"], fill_value=0, margins=True)

sum            mean          
                                          Price Quantity  Price  Quantity
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500  1.000000
                            Maintenance    5000        2   5000  2.000000
                            Software      10000        1  10000  1.000000
              Daniel Hilton CPU          105000        4  52500  2.000000
                            Software      10000        1  10000  1.000000
              John Smith    CPU           35000        1  35000  1.000000
                            Maintenance    5000        2   5000  2.000000
Fred Anderson Cedric Moss   CPU           95000        3  47500  1.500000
                            Maintenance    5000        1   5000  1.000000
                            Software      10000        1  10000  1.000000
              Wendy Yule    CPU          165000        7  82500  3.500000
                            Maintenance    7000        3   7000  3.000000
                            Monitor        5000        2   5000  2.000000
All                                      522000       30  30705  1.764706

Let’s move the analysis up a level and look at our pipeline at the manager level. Notice how the status is ordered based on our earlier category definition.

In [84]:
pd.pivot_table(df, index=["Manager", "Status"], values=["Price"], aggfunc=["sum"], fill_value=0, margins=True)

sum
                          Price
Manager       Status           
Debra Henley  presented   50000
              pending     50000
              declined    70000
              won         65000
Fred Anderson presented   45000
              pending      5000
              declined    65000
              won        172000
All                      522000

A really handy feature is the ability to pass a dictionary to the aggfunc so you can perform different functions on each of the values you select. This has a side-effect of making the labels a little cleaner.

In [85]:
aggregation = {"Quantity": "count","Price" : "sum"}

pd.pivot_table(df, index=["Manager","Status"], columns=["Product"], values=["Quantity","Price"], aggfunc=aggregation,fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  presented   30000           0       0    20000        1   
              pending     40000       10000       0        0        1   
              declined    70000           0       0        0        2   
              won         65000           0       0        0        1   
Fred Anderson presented   30000           0    5000    10000        1   
              pending         0        5000       0        0        0   
              declined    65000           0       0        0        1   
              won        165000        7000       0        0        2   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  presented           0       0        2  
              pending             2       0        0  
              declined            0       0        0  
              won                 0       0        0  
Fred Anderson presented           0       1        1  
              pending             1       0        0  
              declined            0       0        0  
              won                 1       0        0

In [86]:
table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)

table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  presented  30000           0       0    10000   30000   
              pending    40000        5000       0        0   40000   
              declined   35000           0       0        0   70000   
              won        65000           0       0        0   65000   
Fred Anderson presented  30000           0    5000    10000   30000   
              pending        0        5000       0        0       0   
              declined   65000           0       0        0   65000   
              won        82500        7000       0        0  165000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  presented           0       0    20000        1           0   
              pending         10000       0        0        1           2   
              declined            0       0        0        2           0   
              won                 0       0        0        1           0   
Fred Anderson presented           0    5000    10000        1           0   
              pending          5000       0        0        0           1   
              declined            0       0        0        1           0   
              won              7000       0        0        2           1   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  presented       0        2  
              pending         0        0  
              declined        0        0  
              won             0        0  
Fred Anderson presented       1        1  
              pending         0        0  
              declined        0        0  
              won             0        0

# Advanced Pivot Table Filtering

Once you have generated your data, it is in a DataFrame so you can filter on it using your standard DataFrame functions.

If you want to look at just one manager:

In [87]:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley presented  30000           0       0    10000  30000           0   
             pending    40000        5000       0        0  40000       10000   
             declined   35000           0       0        0  70000           0   
             won        65000           0       0        0  65000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley presented       0    20000        1           0       0        2  
             pending         0        0        1           2       0        0  
             declined        0        0        2           0       0        0  
             won             0        0        1           0       0        0

In [88]:
table.query('Status == ["pending","won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  pending  40000        5000       0        0   40000       10000   
              won      65000           0       0        0   65000           0   
Fred Anderson pending      0        5000       0        0       0        5000   
              won      82500        7000       0        0  165000        7000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  pending       0        0        1           2       0        0  
              won           0        0        1           0       0        0  
Fred Anderson pending       0        0        0           1       0        0  
              won           0        0        2           1       0        0